# DSCI 573 - Feature and Model Selection

# Lab 2: Feature engineering, feature selection

## Table of contents
- [Submission instructions](#si) (4%)
- [Exercise 1: Feature engineering](#1) (74%)
- [(Optional) Exercise 2: Change of basis](#2) 
- [Exercise 3: Recursive feature elimination and forward selection](#3) (22%)

## Submission instructions <a name="si"></a>
<hr>
rubric={mechanics:4}

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **Please add a link to your GitHub repository here: [LINK TO REPO](https://github.ubc.ca/mds-2021-22/DSCI_573_lab2_azandian)**
- Be sure to follow the [general lab instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions/).
- Make at least three commits in your lab's GitHub repository.
- Push the final .ipynb file with your solutions to your GitHub repository for this lab.
- Upload the .ipynb file to Gradescope.
- If the .ipynb file is too big or doesn't render on Gradescope for some reason, also upload a pdf or html in addition to the .ipynb. 
- Make sure that your plots/output are rendered properly in Gradescope.

> [Here](https://github.com/UBC-MDS/public/tree/master/rubric) you will find the description of each rubric used in MDS.

> As usual, do not push the data to the repository. 

In [1]:
import os
import string

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import RFE, RFECV
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, Ridge, RidgeCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    ShuffleSplit,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    PolynomialFeatures,
    StandardScaler,
)
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.svm import SVC, SVR

%matplotlib inline

## Exercise 1: Feature engineering <a name="1"></a>
<hr>

One of the most important aspects which influences performance of machine learning models is the features used to represent the problem. If your underlying representation is bad whatever fancy model you use is not going to help. With a better feature representation, a simple and a more interpretable model is likely to perform reasonably well. 

**Feature engineering** is the process of transforming raw data into features that better represent the underlying problem to the predictive models. 

In this exercise we'll engineer our own features on [the Disaster Tweets dataset](https://www.kaggle.com/vstepanenko/disaster-tweets). 

Note that coming up with features is difficult, time-consuming, and requires expert knowledge. The purpose of this exercise is to give you a little taste of feature engineering, which you are likely to be doing in your career as a data scientist or a machine learning practitioner. In this exercise, since we'll be using simplistic features, you might not get better scores with your engineered features, and that's fine. The purpose here is to make you familiar with the process of feature engineering rather than getting the best scores. 

As usual, download the dataset, unzip it and save it in your lab folder. As usual, do not push it into the repository. 

The code below reads the data CSV assuming that the file is stored as `tweets.csv` in your lab folder. 

In [2]:
df = pd.read_csv("tweets.csv", usecols=["keyword", "text", "target", "location"])
train_df, test_df = train_test_split(df, test_size=0.2, random_state=2)
train_df.head()

,keyword,location,text,target
3289,debris,NaN,"Unfortunately, both plans fail as the 3 are im...",0
2672,crash,SLC,I hope this causes Bernie to crash and bern. S...,0
2436,collide,NaN,—pushes himself up from the chair beneath to r...,0
9622,suicide%20bomb,NaN,Widow of CIA agent killed in 2009 Afghanistan ...,1
8999,screaming,Azania,As soon as God say yes they'll be screaming we...,0


In [3]:
X_train, y_train = train_df.drop(columns=["target"]), train_df["target"]
X_test, y_test = test_df.drop(columns=["target"]), test_df["target"]

### 1.1 
rubric={reasoning:6}

**Your tasks:**

1. What is the prediction problem that we are trying to solve here? State in your own words. (One sentence is enough.) 
2. Do we have class imbalance? If yes, do we need to deal with it? Briefly explain. 
3. What metric(s) would you use in this case? Accordingly, define the `scoring_metrics` below. 

> You may decide to use more than one metrics. 

1. We are trying to predict whether the content of a tweet is a real disaster or not.
2. According to the below calculations we see that there is a class imbalance. We definitely need to deal with the class imbalance because the difference between 81% and 18% seems large.
3. Assuming that we use this for a real situations (and we want to send out forces to help victims before it is too late), it is very important to have a high precision meaning that we want to know the number of tweets that we correctly labeled "disaster" from the list of tweets that are reporting a disaster in reality. I will also be using f1 score to make sure I don't get really low recall scores.

In [4]:
y_train.value_counts(normalize=True)

0    0.812995
1    0.187005
Name: target, dtype: float64

In [5]:
scoring_metrics = ["precision", "f1"]

<br><br>

### 1.2 The location feature
rubric={reasoning:6}

The location feature seems quite messy. 

**Your tasks:**

1. Identify at least two challenges that would be involved in encoding the location feature. 
2. It is fine if you drop this feature in this assignment. But if you have to include it, how you might encode it? You don't have to write any code. Just pointing out challenges and providing some reasonable ideas should be fine. 

1. The first challenge would be the high ratio of NA's and the second is that this looks like a free text without any structure. For example in some examples name of a city has been used where in other observations it is a country or state. Also, there is a use of emoticons which should be translated to text.
2. More than a third of the values are unique. In order to reduce the number of unique values, I will use some wrangling and built-in string match functionality of pandas to see if I can replace the name of the cities and states with countries (using list of cities and countries from another dataframe). I will also use a library like spacymoji to translate emoji's to text in order to see if I can get name of any flags translated to country names. After extracting as much country as I could, I will use OneHotEncoder and only provide the categories of the 192 countries. Instead of the last step, I could alternatively use CountVectorizer and use an `n-gram range` of 1 to 3 (because name of some countries is comprised of multiple names) and use `min_df` to ignore the locations that have lower frequency that the threshold that I specify in order to remove the ones that only appear for one or two times.

In [6]:
X_train['location'].nunique()
CountVectorizer()

CountVectorizer()

In [7]:
X_train['location'].unique()[:10]

array([nan, 'SLC', 'Azania', 'United States',
       'Amphoe Mueang Nakhon Ratchasim', 'Accra, Ghana', 'Lagos, Nigeria',
       'Rohnert Park, CA', 'Brighton', 'Hell,Hades,Mictlan,Tartarus'],
      dtype=object)

In [8]:
X_train['location'].value_counts()

United States                     80
Australia                         68
London, England                   66
UK                                62
India                             60
                                  ..
Arizona City, AZ                   1
Yorkshire & Scotland               1
th: hakuna matata                  1
Tacloban City, Eastern Visayas     1
Greater Manchester                 1
Name: location, Length: 3746, dtype: int64

In [9]:
print(f"{round(100 * sum(X_train['location'].isna()) / X_train.shape[0], 1)}% of the values in the location column are missing.")

30.0% of the values in the location column are missing.


<br><br>

### 1.3 Identifying feature types
rubric={accuracy:6,reasoning:2}

**Your tasks:**

In preparation for building a classifier, identify different feature types and set up a column transformer that performs whatever feature transformations you deem sensible. This can include dropping features if you think they are not helpful. In each case, briefly explain your rationale with 1-2 sentences. 

> Hint: Remember that for `CountVectorizer` transformer, you need to pass a 1-D array or a pandas.Series. So in a column transformer, you pass a string rather than a list of features for this transformer. [Here](https://pages.github.ubc.ca/mds-2021-22/DSCI_571_sup-learn-1_students/lectures/05_text-feats.html#demo-of-incorporating-text-features) you'll find an example of incorporating text column in a column transformer. 

> It's fine if you drop the location feature for this assignment. 

- I am dropping the location column as it does not provide any useful information and is in a free text format.
- I will be watching that my number of features won't grow more than my observations (m < n). I will use a Countvectorizer for the `text` column and will use max_feature to limit the number of columns. If not specified, it will give me more than 23,000 feature columns.
- For the `keyword` column I will use a OneHotEncoder as it is a categorical column with 200 unique values.

In [10]:
X_train.head()

,keyword,location,text
3289,debris,NaN,"Unfortunately, both plans fail as the 3 are im..."
2672,crash,SLC,I hope this causes Bernie to crash and bern. S...
2436,collide,NaN,—pushes himself up from the chair beneath to r...
9622,suicide%20bomb,NaN,Widow of CIA agent killed in 2009 Afghanistan ...
8999,screaming,Azania,As soon as God say yes they'll be screaming we...


In [11]:
drop = ["location"]
text_feature = "text"  # Inputig as string rather than list
categorical_feature = ["keyword"]

In [12]:
preprocessor = make_column_transformer(
    (CountVectorizer(stop_words="english", max_features=3000), text_feature),
    (OneHotEncoder(handle_unknown="ignore",sparse=False), categorical_feature)
)

In [13]:
preprocessor.fit_transform(X_train)

<9096x3219 sparse matrix of type '<class 'numpy.float64'>'
	with 68911 stored elements in Compressed Sparse Row format>

In [14]:
preprocessor.named_transformers_["countvectorizer"].get_feature_names_out()

array(['00', '000', '01', ..., 'zip', 'zone', '하윤빈'], dtype=object)

<br><br>

### 1.4 DummyClassifier
rubric={accuracy:2}

**Your tasks:**

1. Report cross-validation scores for `DummyClassifier` using `scoring_metrics` you defined in 1.1. 

> You might want to use the `results` dictionary and `mean_std_cross_val_scores` function below to organize your results.

In [15]:
results = {}

In [16]:
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

In [17]:
dummy_pipe = make_pipeline(preprocessor, DummyClassifier())

results["Dummy Classifier"] = mean_std_cross_val_scores(
    dummy_pipe, X_train, y_train, scoring=scoring_metrics
)
pd.DataFrame(results)

C:\Users\artan\miniconda3\envs\573\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\artan\miniconda3\envs\573\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\artan\miniconda3\envs\573\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\artan\miniconda3\envs\573\lib\site-packages\sklearn\metrics\_classification.py:

,Dummy Classifier
fit_time,0.172 (+/- 0.011)
score_time,0.043 (+/- 0.008)
test_precision,0.000 (+/- 0.000)
test_f1,0.000 (+/- 0.000)


<br><br>

### 1.5 Logistic regression
rubric={accuracy:3}

**Your tasks:**
1. Now try logistic regression classifier with the same scoring metrics and default hyperparameters. 

In [18]:
lr_pipe = make_pipeline(preprocessor, LogisticRegression())
results["Logistic Regression"] = mean_std_cross_val_scores(
    lr_pipe, X_train, y_train, scoring=scoring_metrics
)
pd.DataFrame(results)

,Dummy Classifier,Logistic Regression
fit_time,0.172 (+/- 0.011),0.256 (+/- 0.017)
score_time,0.043 (+/- 0.008),0.041 (+/- 0.008)
test_precision,0.000 (+/- 0.000),0.771 (+/- 0.027)
test_f1,0.000 (+/- 0.000),0.619 (+/- 0.026)


<br><br>

### 1.6 Hyperparameter optimization 
rubric={accuracy:7}

**Your tasks:**
1. Jointly tune hyperparameters of logistic regression and `CountVectorizer`. Some options for hyperparameters are:   
    - `C` of logistic regression
    - `class_weight` of logistic regression
    - `max_features` of `CountVectorizer`
2. Report the best hyperparameter values and best cross-validation scores.

> Hint: You can access `max_features` of `CounVectorizer` of a `ColumnTransformer` object as: `columntransformer__countvectorizer__max_features`

In [19]:
param_distributions = {
    "logisticregression__C": np.logspace(-2, 2, 5),
    "logisticregression__class_weight": ["balanced", None],
    "columntransformer__countvectorizer__max_features": np.linspace(
        1000, 5000, 9, dtype=int
    ),
}

hyper_opt = RandomizedSearchCV(
    estimator=lr_pipe,
    param_distributions=param_distributions,
    scoring="f1",
    n_iter=20,
    cv=5,
    n_jobs=-1,
)

hyper_opt.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('countvectorizer',
                                                                               CountVectorizer(max_features=3000,
                                                                                               stop_words='english'),
                                                                               'text'),
                                                                              ('onehotencoder',
                                                                               OneHotEncoder(handle_unknown='ignore',
                                                                                             sparse=False),
                                                                               ['keyword'])])),
                                             ('logisticregression',
    

In [20]:
hyper_opt.best_params_

{'logisticregression__class_weight': 'balanced',
 'logisticregression__C': 1.0,
 'columntransformer__countvectorizer__max_features': 5000}

In [21]:
print(f"best precision score is {round(hyper_opt.best_score_, 3)}.")

best precision score is 0.65.


<br><br>

### Exercise 1.7: Feature engineering
rubric={accuracy:10,reasoning:8}

Is it possible to further improve the scores? How about adding new features based on our intuitions? In this exercise, you will be extracting your own features that might be useful for this prediction task. In other words, you will carry out **feature engineering**. 

The code below adds some very basic length-related and sentiment features. We will be using a popular library called `nltk` for this exercise. If you have successfully created the course `conda` environment on your machine, you should already have this package in the environment.  

**Your tasks:**
1. Run the starter code below which creates three new features: 
    - Relative character length in the tweet. 
    - Number of words in the tweet.
    - Sentiment of the tweet. In particular, we'll be using a metric called "compound score" representing the sentiment in the given tweet. (A score of -1 corresponds to most extreme negative and a score of +1 corresponds to most extreme positive.) This score is extracted using [Vader lexicon](https://github.com/cjhutto/vaderSentiment). In 571, we trained our own models for sentiment analysis. Here we are using some **pre-trained model** to extract sentiment expressed in the tweets. Below I am showing you a couple of examples of using using this pre-trained model for getting sentiment information on some random sentences. 
2. Extract at least two more features that you think might be relevant for prediction and store them as new columns in the train and test sets. Briefly explain your intuition on why these features might help the prediction task. 
3. Would it have been OK to create new columns directly in the original `df` instead of creating them separately for train and test splits? Would that be violation of the golden rule? 

> As I mentioned in the lecture, coming up with good features is not easy. It can be time consuming, as you need to think deeply about what representation would help the prediction problem. But please don't spend WAY too much time on this question. (Of course if you're having fun you're welcome to spend as much time as you want!) My suggestion would be coming up with some simple features first, getting the pipeline working, and completing the assignment. If you have time after submitting all your assignments, you can always come back to this question. 

In [22]:
import nltk

nltk.download("vader_lexicon")
nltk.download("punkt")
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\artan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\artan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [23]:
s = "MDS students are smart, sweet, and funny."
print(sid.polarity_scores(s))

{'neg': 0.0, 'neu': 0.317, 'pos': 0.683, 'compound': 0.8225}


In [24]:
s = "MDS students are tired because of all the hard work they have been doing."
print(sid.polarity_scores(s))

{'neg': 0.264, 'neu': 0.736, 'pos': 0.0, 'compound': -0.5106}


In [25]:
def get_relative_length(text, TWITTER_ALLOWED_CHARS=280.0):
    """
    Returns the relative length of text.

    Parameters:
    ------
    text: (str)
    the input text

    Keyword arguments:
    ------
    TWITTER_ALLOWED_CHARS: (float)
    the denominator for finding relative length

    Returns:
    -------
    relative length of text: (float)

    """
    return len(text) / TWITTER_ALLOWED_CHARS


def get_length_in_words(text):
    """
    Returns the length of the text in words.

    Parameters:
    ------
    text: (str)
    the input text

    Returns:
    -------
    length of tokenized text: (int)

    """
    return len(nltk.word_tokenize(text))


def get_sentiment(text):
    """
    Returns the compound score representing the sentiment of the given text: -1 (most extreme negative) and +1 (most extreme positive)
    The compound score is a normalized score calculated by summing the valence scores of each word in the lexicon.

    Parameters:
    ------
    text: (str)
    the input text

    Returns:
    -------
    sentiment of the text: (str)
    """
    scores = sid.polarity_scores(text)
    return scores["compound"]

In [26]:
train_df = train_df.assign(n_words=train_df["text"].apply(get_length_in_words))
train_df = train_df.assign(vader_sentiment=train_df["text"].apply(get_sentiment))
train_df = train_df.assign(rel_char_len=train_df["text"].apply(get_relative_length))

test_df = test_df.assign(n_words=test_df["text"].apply(get_length_in_words))
test_df = test_df.assign(vader_sentiment=test_df["text"].apply(get_sentiment))
test_df = test_df.assign(rel_char_len=test_df["text"].apply(get_relative_length))

In [27]:
train_df["news_agency"] = train_df["text"].apply(lambda x: "http" in x)
train_df["hashtag"] = train_df["text"].apply(lambda x: "#" in x)
train_df["Iran"] = train_df["text"].apply(lambda x: "iran" in x.lower())

test_df["news_agency"] = test_df["text"].apply(lambda x: "http" in x)
test_df["hashtag"] = test_df["text"].apply(lambda x: "#" in x)
test_df["Iran"] = test_df["text"].apply(lambda x: "iran" in x.lower())

train_df.head()

,keyword,location,text,target,n_words,vader_sentiment,rel_char_len,news_agency,hashtag,Iran
3289,debris,NaN,"Unfortunately, both plans fail as the 3 are im...",0,22,-0.7650,0.425000,False,False,False
2672,crash,SLC,I hope this causes Bernie to crash and bern. S...,0,18,-0.5697,0.267857,True,False,False
2436,collide,NaN,—pushes himself up from the chair beneath to r...,0,21,0.0000,0.439286,True,False,False
9622,suicide%20bomb,NaN,Widow of CIA agent killed in 2009 Afghanistan ...,1,20,-0.9460,0.428571,True,False,False
8999,screaming,Azania,As soon as God say yes they'll be screaming we...,0,14,0.2960,0.203571,False,False,False


_2. I am extracting whether a news agency has been mentioned in the tweet by seeing if there is a link (https) to news agency. I have also checked if a hashtag has been used to make a certain news a hashtag. Also, as an Iranian there has always been disaster associated with the name Iran through recent history. I will give this a try to see if it really makes sense._  
_3. As long as the created features are derived by applying a function on individual row and not the rest of the column we are not violating the Golden rule. For the case of this exercise, since the n_words, vader sentiment, and rel_char_len are applied on single values not considering the rest of the observations, we are not violating the golden rule._

<br><br>

### 1.8 Pipeline with all features
rubric={accuracy:6}

**Your tasks:**
1. Define a new column transformer for your mixed feature types in your new data set with the features you have created above in 1.7.
2. Define a pipeline with logistic regression, setting the hyperparameter values to the ones you found in 1.6. 
3. Report cross-validation scores with this new pipeline. 

> It's possible that you might get different hyperparameter values with these new features. In real life, you might have to go through a couple of iterations of hyperparameter tuning and feature engineering. But in this assignment, in the interest of time, we won't carry out hyperparameter optimization again after feature engineering.  

In [28]:
X_train_new = train_df.drop("target", axis=1)
y_train_new = train_df["target"]

X_test_new = test_df.drop("target", axis=1)
y_test_new = test_df["target"]

In [29]:
text_feature = "text"
binary_features = ["news_agency", "hashtag", "Iran"]
categorical_features = ["keyword"]
numeric_feature = ["n_words"]
passthrough = ["vader_sentiment", "rel_char_len"]
drop_feature = ["location"]


In [30]:
NLP_preprocessor = make_column_transformer(
    (
        CountVectorizer(
            stop_words="english",
            max_features=hyper_opt.best_params_[
                "columntransformer__countvectorizer__max_features"
            ],
        ),
        text_feature,
    ),
    (OneHotEncoder(drop="if_binary"), binary_features),
    (OneHotEncoder(handle_unknown="ignore"), categorical_features),
    (StandardScaler(), numeric_feature),
    ("passthrough", passthrough),
)

In [31]:
hyper_opt.best_params_

{'logisticregression__class_weight': 'balanced',
 'logisticregression__C': 1.0,
 'columntransformer__countvectorizer__max_features': 5000}

In [32]:
NLP_pipeline = make_pipeline(
    NLP_preprocessor,
    LogisticRegression(
        C=hyper_opt.best_params_["logisticregression__C"],
        class_weight=hyper_opt.best_params_["logisticregression__class_weight"],
        max_iter=300
    ),
)

In [33]:
mean_std_cross_val_scores(
    NLP_pipeline,
    X_train_new,
    y_train_new,
    scoring=scoring_metrics,
    cv=5,
)

fit_time          0.459 (+/- 0.027)
score_time        0.071 (+/- 0.009)
test_precision    0.624 (+/- 0.008)
test_f1           0.658 (+/- 0.018)
dtype: object

<br><br>

### 1.9 Interpretation
rubric={accuracy:10,reasoning:4}

1. Do you see any improvements in the scores with the new features? If you do not see big improvements in scores with new features, that's OK. Do not get discouraged. Feature engineering is hard and requires domain expertise. The purpose of this exercise is to make you familiar to the process of extracting new features rather than getting the best scores. 
2. Show first few (e.g., first 10) most important features and their coefficients identified by the model. 
3. Examine the coefficients of the features we have extracted above. Do these coefficients align with your intuitions? Briefly explain. 

> Hint: You need to fit the pipeline in order to get feature names from your transformers. 

> Assuming that your pipeline is named `pipe_lr` and it's fitted, you can get the feature names created by `CountVectorizer` using the syntax below: 

```pipe_lr.named_steps["columntransformer"].named_transformers_["countvectorizer"].get_feature_names_out()```

_1. We see slight improvement on the f1 score but not much._

In [34]:
NLP_pipeline.fit(X_train_new, y_train_new)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('countvectorizer',
                                                  CountVectorizer(max_features=5000,
                                                                  stop_words='english'),
                                                  'text'),
                                                 ('onehotencoder-1',
                                                  OneHotEncoder(drop='if_binary'),
                                                  ['news_agency', 'hashtag',
                                                   'Iran']),
                                                 ('onehotencoder-2',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['keyword']),
                                                 ('standardscaler',
                                                  StandardScaler(),
           

In [35]:
text_features = (
    NLP_pipeline.named_steps["columntransformer"]
    .named_transformers_["countvectorizer"]
    .get_feature_names_out()
    .tolist()
)
keyword_features = (
    NLP_pipeline.named_steps["columntransformer"]
    .named_transformers_["onehotencoder-2"]
    .get_feature_names_out()
    .tolist()
)

column_names = (
    text_features + binary_features + keyword_features + numeric_feature + passthrough
)

coefficients = NLP_pipeline.named_steps["logisticregression"].coef_.flatten()

In [36]:
coef_df = pd.DataFrame({"feature": column_names, "coefficient": coefficients}).sort_values("coefficient", ascending=False)
coef_df[:10]

,feature,coefficient
5224,rel_char_len,2.417235
4463,thunderstorm,2.263189
5216,keyword_windstorm,2.089528
4295,survived,2.079450
2116,influenza,1.966293
3678,road,1.941573
4625,ukrainian,1.894293
3623,rescued,1.866336
1144,died,1.858740
1145,dies,1.818639


In [37]:
coef_df[-10:]

,feature,coefficient
4788,waiting,-1.306888
2664,mean,-1.320104
5022,keyword_blazing,-1.353575
1216,don,-1.386778
1897,heart,-1.408779
5069,keyword_demolition,-1.416988
308,ass,-1.456028
4578,trudeau,-1.456181
2551,love,-1.521717
5048,keyword_collapse,-1.620347


_From the above top ranking positive and negative coefficients, we can see that most of them make sense._
_Below are the coefficients for the three columns that I added. Two of them have high positive coefficients, but the feature capturing the news agencies actually has a negative coefficient which might probably be due to the fact that not all news are disaster related (or that I have not used the proper method to extract news related tweets!)._

_Resultant coefficient from added feature `Iran`:_

In [38]:
coef_df.query("feature == 'Iran'")

,feature,coefficient
5002,Iran,0.746721


_Resultant coefficient from added feature `hashtag`:_

In [39]:
coef_df.query("feature == 'hashtag'")

,feature,coefficient
5001,hashtag,0.3752


_Resultant coefficient from added feature `news_agency`:_

In [40]:
coef_df.query("feature == 'news_agency'")

,feature,coefficient
5000,news_agency,-0.460113


<br><br>

### 1.10 Test results
rubric={reasoning:4}

**Yout tasks**

1. Report scores with your `scoring_metrics` on the test set. Use the model trained with all features. 

In [41]:
from sklearn.metrics import f1_score
test_score = f1_score(y_test_new, NLP_pipeline.predict(X_test_new))
print(f"The f1 score for the test data is {round(test_score, 2)}.")

The f1 score for the test data is 0.7.


<br><br><br><br>

## Dataset for next exercises
<hr>

In the following exercises, we'll be using [`sklearn`'s boston housing dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html). 

_I just noticed the warning that this dataset has an ethical problem. We'll use this dataset for this assignment, as I didn't get a chance to replace it. But I encourage you to read why the dataset has an ethical problem and think about what would be the consequences of using this dataset._

In [42]:
from sklearn.datasets import load_boston

boston_housing = load_boston()
print(boston_housing.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

C:\Users\artan\miniconda3\envs\573\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing

In [43]:
boston_df = pd.DataFrame(boston_housing.data, columns=boston_housing.feature_names)
boston_df["target"] = boston_housing.target
train_df, test_df = train_test_split(boston_df, test_size=0.2, random_state=2)

X_train, y_train = train_df.drop(columns=["target"]), train_df["target"]
X_test, y_test = test_df.drop(columns=["target"]), test_df["target"]

<br><br>

## (Optional) Exercise 2: Change of basis <a name="2"></a>
<hr>

rubric={reasoning:1}

The linear model is problematic when the target is a non-linear function of the input. With high dimensional data we cannot really know whether the target is a linear or non-linear function of the input. One way to examine this is by using _polynomial features_. Suppose you have a single feature $x_1$ in your original data, you can think of transforming the data into the following matrix $X_{poly}$ where each of its rows contains the values $(X_{i})^j$ for $j=0$ up to some maximum $degree$. E.g., 

$$
X_{poly} = \left[\begin{array}{cccc}
1 & x_1 & (x_1)^2 & (x_1)^3\\
1 & x_2 & (x_2)^2 & (x_2)^3\\
\vdots\\
1 & x_n & (x_n)^2 & (x_N)^3\\
\end{array}
\right],
$$

We can then fit a least squares model as if the above were our data set. You can think of this as "changing the model by changing the data" since we are still using a linear model but making the fit nonlinear by inventing new features. 

**Your tasks:**
1. Is it possible to visualize the Boston housing data and examine whether a linear fit is good fit for this dataset or not? 
2. Carry out cross-validation using `DummyRegressor` on the train portion. 
3. Define a pipeline with `PolynomialFeatures` and `RidgeCV`. 
4. Examine the train and validation scores for three values for `degree` hyperparameter of `PolynomialFeatures`: 1, 2, and 3. Use either negative MAPE or `neg_root_mean_squared_error` for scoring. 
5. Which value of `degree` is giving you the best results? How many new features do you have with this degree?

<br><br><br><br>

## Exercise 3: Feature importances and feature selection <a name="3"></a>
<hr>

In this exercise we'll briefly explore feature importances, recursive feature elimination, adding polynomial features in a pipeline, and forward/backward selection. You could use the scoring method of your choice. The default $R^2$ is fine too.  

### Exercise 3.1 Adding random noise
rubric={reasoning:4}

The following code shows the coefficients learned by `RidgeCV` on the Boston housing dataset. It then adds a column of random noise to `X_train` and re-trains and examines the coefficients again. We see that the model has assigned a non-zero coefficient to the noise feature. But wait, we know this feature can't possibly be useful. 

**Yout taks:**

1. Why is the importance of the random noise feature non-zero (and in fact larger than for some real features)? Maximum 2 sentences.

In [44]:
lrcv = RidgeCV()
lrcv.fit(X_train, y_train)
pd.DataFrame(data=lrcv.coef_, index=X_train.columns, columns=["coefficient"])

,coefficient
CRIM,-0.107895
ZN,0.039087
INDUS,-0.020005
CHAS,3.138096
NOX,-15.339384
RM,3.640792
AGE,0.008323
DIS,-1.367897
RAD,0.321224
TAX,-0.011728


In [45]:
random_noise = np.random.randn(X_train.shape[0], 1)
X_train_noise = pd.concat(
    (X_train, pd.DataFrame(random_noise, columns=["noise"], index=X_train.index)),
    axis=1,
)
X_train_noise.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,noise
321,0.18159,0.0,7.38,0.0,0.493,6.376,54.3,4.5404,5.0,287.0,19.6,396.90,6.87,-0.494632
37,0.08014,0.0,5.96,0.0,0.499,5.850,41.5,3.9342,5.0,279.0,19.2,396.90,8.77,0.843134
286,0.01965,80.0,1.76,0.0,0.385,6.230,31.5,9.0892,1.0,241.0,18.2,341.60,12.93,1.466356
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,-1.337690
25,0.84054,0.0,8.14,0.0,0.538,5.599,85.7,4.4546,4.0,307.0,21.0,303.42,16.51,-0.426990


In [46]:
lrcv = RidgeCV()
lrcv.fit(X_train_noise, y_train)
pd.DataFrame(data=lrcv.coef_, index=X_train_noise.columns, columns=["coefficient"])

,coefficient
CRIM,-0.107995
ZN,0.039140
INDUS,-0.017164
CHAS,3.154169
NOX,-15.359460
RM,3.645749
AGE,0.008643
DIS,-1.366460
RAD,0.321421
TAX,-0.011852


_Column random noise is as the name suggests random values from a normal distribution with mean of 0 and standard deviation of 1. In fact, the values of this column are in the range of what other numeric columns would become after applying StandardScaler. The high value of coefficient is mainly due to confounding where some variations are not truly defined, and when we introduce the random feature with some values, it fills the gap by trying to make that noise variable relevant._

In [47]:
X_train_noise["AGE"].value_counts()

100.0    30
98.2      4
87.9      4
97.9      4
98.8      4
         ..
58.1      1
49.0      1
94.0      1
45.4      1
56.5      1
Name: AGE, Length: 304, dtype: int64

<br><br>

### 3.2 `RFECV` 
rubric={accuracy:4}

In this exercise, you'll explore recursive feature elimination for feature selection. The code below defines a pipeline with feature selection incorporated in it. `RFECV` is used for feature selection in the pipeline; it uses cross-validation to decide how many features to select. The selected features are passed to `RidgeCV`, which has built-in cross-validation to tune the `alpha` hyperparameter.  

**Your tasks:**

1. Carry out cross-validation using the pipeline below.  
2. How many features have been selected by `RFECV`? You can access this information using the `n_features_` attribute of the `RFECV` step from the pipeline. 

In [48]:
pipe_rfe_ridgecv = make_pipeline(StandardScaler(), RFECV(Ridge(), cv=10), RidgeCV())

In [49]:
rfe_score = mean_std_cross_val_scores(
    pipe_rfe_ridgecv, X_train, y_train, cv=5, return_train_score=True
)
rfe_score

fit_time       0.188 (+/- 0.055)
score_time     0.002 (+/- 0.001)
test_score     0.704 (+/- 0.077)
train_score    0.730 (+/- 0.017)
dtype: object

In [50]:
rfe_score["test_score"]

'0.704 (+/- 0.077)'

In [51]:
pipe_rfe_ridgecv.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('rfecv', RFECV(cv=10, estimator=Ridge())),
                ('ridgecv', RidgeCV(alphas=array([ 0.1,  1. , 10. ])))])

In [52]:
pipe_rfe_ridgecv.named_steps["rfecv"].n_features_

11

<br><br>

### 3.3: `PolynomialFeatures` + [`RFECV`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html)
rubric={accuracy:4,reasoning:3}

**Your tasks:**

1. Add one more step to the pipeline above, `PolynomialFeatures()`, for extracting polynomial features. 
2. Carry out cross-validation using the pipeline and report the mean validation scores. 
3. Are you getting better scores compared to 3.2? If yes, what might be the reason?   

In [53]:
poly_pipe = make_pipeline(
    StandardScaler(), PolynomialFeatures(degree=3), RFECV(Ridge(), cv=10), RidgeCV()
)

In [54]:
poly_score = mean_std_cross_val_scores(
    poly_pipe, X_train, y_train, cv=5, return_train_score=True
)
poly_score

fit_time       23.586 (+/- 2.701)
score_time      0.003 (+/- 0.001)
test_score      0.845 (+/- 0.058)
train_score     0.925 (+/- 0.017)
dtype: object

_Our mean crossvalidation R2 score improved a lot from 0.7 to 0.84. This is mainly because of the higher number of features and the fact that some of these features have had a polynomial shape and once transformed to third degree polynomial, we have been able to fit them better in our regression model._

<br><br>

### (Optional) 3.4 
rubric={reasoning:1}

**Your tasks:**

1. How many total features there will be after applying `PolynomialFeatures` transformation? 
2. How many features have been selected by `RFECV`? 
3. Show the feature names of the features selected by `RFECV`. 

In [55]:
poly_pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('rfecv', RFECV(cv=10, estimator=Ridge())),
                ('ridgecv', RidgeCV(alphas=array([ 0.1,  1. , 10. ])))])

In [56]:
poly_pipe.named_steps["rfecv"].n_features_in_

560

In [57]:
poly_pipe.named_steps["rfecv"].n_features_

40

In [58]:
poly_pipe.named_steps["rfecv"].get_feature_names_out()

array(['x6', 'x7', 'x8', 'x35', 'x63', 'x64', 'x74', 'x76', 'x84', 'x89',
       'x90', 'x95', 'x99', 'x134', 'x145', 'x167', 'x169', 'x187',
       'x195', 'x251', 'x284', 'x336', 'x403', 'x420', 'x421', 'x423',
       'x424', 'x453', 'x463', 'x484', 'x496', 'x505', 'x514', 'x515',
       'x518', 'x524', 'x525', 'x541', 'x542', 'x544'], dtype=object)

<br><br>

### 3.5: `PolynomialFeatures` + backward selection
rubric={accuracy:4,reasoning:3}

**Your tasks:**
1. Now define a pipeline with `StandardScaler()`, `PolynomialFeatures()`, and backward feature selection (`SequentialFeatureSelector` with `Ridge` and direction `backward`) and report cross-validation scores. 
2. Comment on the scores. 
3. What is the difference between RFE and backward selection? 

In [59]:
poly_seq_pipe = make_pipeline(
    StandardScaler(),
    PolynomialFeatures(),
    SequentialFeatureSelector(Ridge(), direction="backward", n_jobs=-1),
    RidgeCV(),
)

In [ ]:
mean_std_cross_val_scores(poly_seq_pipe, X_train, y_train, return_train_score=True)

In [ ]:
poly_seq_pipe.named_steps['sequentialfeatureselector']

_2. The mean crossvalidation score from this pipeline with backward selection are less than the ones from RFECV._

_3. It takes a very longer time to fit a Sequential Feature Selector, and therefore, RFE could be more reliable with larger number of features. Sequential Feature Selector by default it's going to select $d/2$ features, but RFE could continue further and select fewer features which in turn would speed up the score time._  
_In SFS we start with a set of all the features, sequentially find the one feature that has the least effect on the maximization of a cross-validation score and greedily remove features from the set. On the other hand, he goal of RFE is to select features by recursively considering smaller and smaller sets of features instead of just one feature removed by SFS. First, the estimator is trained on the initial set of features and the importance of each feature is obtained either through any specific attribute. Then, the least important features are removed from current set of features. That procedure is recursively repeated on the new set until the desired number of features to select is eventually reached._

<br><br><br><br>

**PLEASE READ BEFORE YOU SUBMIT:** 

When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from "1" will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Push all your work to your GitHub lab repository. 
4. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 
5. Make sure that the plots and output are rendered properly in your submitted file. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb so that the TAs can view your submission on Gradescope. 

Congratulations on finishing the homework! This was a tricky one but I hope you had fun working on it. Well done :clap:! 


In [ ]:
from IPython.display import Image

Image("eva-well-done.png")